## Simple CNN

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [26]:
# 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 4)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [27]:
# 데이터셋 및 데이터 로더 설정
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
train_dataset = datasets.ImageFolder(root='E:/PycharmProjects/peslab/GDSC SC/recycle_img/Training', transform=transform)
test_dataset = datasets.ImageFolder(root='E:/PycharmProjects/peslab/GDSC SC/recycle_img/Validation', transform=transform)

In [28]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [29]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    print("GPU is available.")
    print("Current CUDA Device:", torch.cuda.current_device())
    print("CUDA Device Name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("GPU is not available, using CPU.")


GPU is available.
Current CUDA Device: 0
CUDA Device Name: NVIDIA GeForce RTX 3060


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [31]:
"""# 모델, 손실 함수, 최적화 알고리즘 설정
model = SimpleCNN().to(device)  # 모델을 GPU로 이동
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 훈련 루프
for epoch in range(10): 
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{10}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
"""

"# 모델, 손실 함수, 최적화 알고리즘 설정\nmodel = SimpleCNN().to(device)  # 모델을 GPU로 이동\ncriterion = nn.CrossEntropyLoss()\noptimizer = optim.Adam(model.parameters(), lr=0.001)\n\n# 훈련 루프\nfor epoch in range(10): \n    for i, (inputs, labels) in enumerate(train_loader):\n        inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU로 이동\n\n        optimizer.zero_grad()\n        outputs = model(inputs)\n        loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.step()\n\n        if (i+1) % 100 == 0:\n            print(f'Epoch [{epoch+1}/{10}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')\n"

In [32]:
model = SimpleCNN().to(device)  # 모델을 GPU로 이동
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [33]:
import pandas as pd
cnn_loss_df = pd.DataFrame(columns=['iteration','num', 'loss'])

In [34]:
num_epochs = 1 # 학습 에폭 수 설정
df_num = 1

num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    s = 0
    n = 1
    for inputs, labels in train_loader:
        # inputs = inputs.unsqueeze(0).expand(3, -1, -1, -1, -1)
        inputs, labels = inputs.to(device), labels.to(device)
        print(inputs.shape)
        # inputs = inputs.permute(0, 2, 1, 3, 4)
        optimizer.zero_grad()
        print(labels)
        # print(labels.shape)
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        # loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        s += len(train_loader)
        print("Completed : ", n, "The rest :", len(train_loader)-n)
        n += 1

        cnn_loss_df.loc[df_num]=[epoch+1, n, str(loss.item())]
        df_num += 1
print("Training complete!")

torch.Size([64, 3, 64, 64])
tensor([3, 0, 3, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2,
        3, 2, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 2, 3, 2, 2, 2, 3, 3, 3,
        2, 2, 3, 3, 3, 2, 1, 2, 3, 2, 2, 2, 2, 3, 1, 3], device='cuda:0')
Completed :  1 The rest : 729
torch.Size([64, 3, 64, 64])
tensor([2, 3, 0, 0, 2, 3, 2, 3, 2, 2, 0, 2, 2, 1, 3, 2, 3, 3, 3, 1, 3, 3, 3, 3,
        3, 2, 2, 3, 3, 3, 2, 3, 3, 3, 2, 2, 2, 0, 0, 0, 3, 3, 2, 2, 2, 3, 3, 3,
        2, 3, 3, 0, 0, 2, 2, 2, 3, 2, 3, 3, 2, 3, 0, 3], device='cuda:0')
Completed :  2 The rest : 728
torch.Size([64, 3, 64, 64])
tensor([2, 3, 3, 3, 2, 3, 2, 3, 0, 3, 2, 3, 3, 0, 2, 3, 3, 3, 3, 3, 2, 1, 3, 3,
        2, 0, 2, 2, 0, 3, 2, 2, 3, 3, 2, 3, 3, 1, 2, 3, 3, 2, 2, 2, 2, 3, 3, 3,
        1, 3, 3, 3, 3, 2, 3, 0, 3, 2, 3, 1, 3, 2, 2, 3], device='cuda:0')
Completed :  3 The rest : 727
torch.Size([64, 3, 64, 64])
tensor([3, 2, 2, 3, 3, 3, 3, 0, 3, 3, 2, 1, 2, 0, 3, 2, 3, 2, 3, 3, 3, 3, 3, 2,
        3, 0, 2,

In [35]:
cnn_loss_df.to_csv("E:/PycharmProjects/peslab/GDSC SC/cnn_recycle_loss_epoch4.csv", index=False)

In [36]:
# 모델 평가
def evaluate_model(loader, model):
    model.eval()  # 모델을 평가 모드로 전환
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            print(labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy

# 테스트 데이터로 모델 평가
test_accuracy = evaluate_model(test_loader, model)
print(f"Test Accuracy: {test_accuracy:.2f}%")

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [37]:
save_path ='E:/PycharmProjects/peslab/GDSC SC/cnn_recycle_epoch4_weights.pth'
torch.save(model.state_dict(), save_path)
print("Model has been saved.")

Model has been saved.


## tflite 모델 저장하기

In [57]:
import torch.onnx
import torch
# 모델이 예상하는 입력 크기에 맞게 입력 데이터 생성
dummy_input = torch.randn(1, 3, 128, 128).cuda()
  # 입력 데이터를 GPU로 이동
model = model.cuda()  # 모델을 GPU로 이동

In [59]:
import onnx
from onnx_tf.backend import prepare

# ONNX 모델 불러오기
onnx_model = onnx.load("simplecnn.onnx")

In [63]:
import onnx
from onnx import numpy_helper

# ONNX 모델 불러오기
onnx_model = onnx.load("simplecnn.onnx")

# ONNX 모델의 입력 이름 수정
for i, input_node in enumerate(onnx_model.graph.input):
    input_node.name = input_node.name.replace(".", "_")

# 수정된 ONNX 모델을 저장
onnx.save(onnx_model, "simplecnn_modified.onnx")


## 실제 사진 한장을 넣었을때 결과물 확인하기 

In [73]:
import torch
from torchvision import transforms
from PIL import Image

# 모델 정의 (주어진 코드와 동일)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 4)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 데이터셋 및 데이터 로더 설정 (주어진 코드와 동일)
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# 특정 이미지 불러오기 및 전처리
image_path = 'C:/Users/20161/Downloads/페트병.png'
image = Image.open(image_path)
image = image.convert('RGB')  # 이미지의 채널 수를 3으로 변경
image = transform(image)  # 이미지를 모델과 동일한 전처리로 변환

# 모델 초기화 및 가중치 불러오기
model = SimpleCNN()
model.load_state_dict(torch.load('E:/PycharmProjects/peslab/GDSC SC/cnn_recycle_weights.pth'))  # 학습된 모델의 가중치 불러오기

# 모델을 evaluation 모드로 설정
model.eval()

# 이미지를 모델에 전달하여 결과 예측
with torch.no_grad():
    output = model(image.unsqueeze(0))  # 배치 차원 추가

# 결과 출력
print(output)

# 예측된 클래스 인덱스 얻기 (가장 높은 확률 값을 가진 클래스)
predicted_class = torch.argmax(output, dim=1).item()
print("Predicted Class Index:", predicted_class)


tensor([[-4.0417, -1.1628,  1.6475,  2.1105]])
Predicted Class Index: 3


## training 기준으로 0-> 건전지, 1-> 스티로폼, 2-> 유리병_투명, 3-> 페트병_무색 